<a href="https://colab.research.google.com/github/chiffonng/jupyter-notebooks/blob/main/get_all_comments_across_notebooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Supply either this list with google drive files
urls = []

# OR the folder to search for every .ipynb notebook that match
FOLDER_URL      = "https://drive.google.com/drive/u/0/folders/1Q6AX1pzb__GR9Vy_Ttw5_yCTlmDAT45H"
INCLUDE_KWDS    = []       # names must include any of these (OR logic)
EXCLUDE_KWDS    = ["pre-class", "PCW", "comments"]      # names must NOT include any of these (AND logic)


# Google Sheet link to store comments queried
TARGET_SHEET_URL = "https://docs.google.com/spreadsheets/d/1V-huGRgju495WbUBchVC_XBUD--YDTPHHq6b0GaTJxY/edit?gid=1638150159#gid=1638150159"

TARGET_RANGE     = "Comments!A:E"

In [6]:
%%capture
#@title: Install dependencies
!uv pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

import re
import google.auth
import google_auth_httplib2
import httplib2

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate and build clients
auth.authenticate_user()
creds, _ = google.auth.default(scopes=[
    'https://www.googleapis.com/auth/drive.metadata.readonly',
    'https://www.googleapis.com/auth/drive.readonly',
    'https://www.googleapis.com/auth/spreadsheets'
])

# Get timeout
http_with_timeout = httplib2.Http(timeout=60)
authed_http = google_auth_httplib2.AuthorizedHttp(creds, http=http_with_timeout)

# Build GDrive and GSheet services
drive_service  = build('drive', 'v3', credentials=creds)
sheets_service = build('sheets', 'v4', credentials=creds)

In [7]:
def extract_folder_id(url: str) -> str:
    """Pull the Google Drive folder ID out of any share URL."""
    patterns = [
        r"/folders/([A-Za-z0-9_-]+)",
        r"[?&]id=([A-Za-z0-9_-]+)"
    ]
    for pat in patterns:
        m = re.search(pat, url)
        if m:
            return m.group(1)
    raise ValueError(f"Could not parse folder ID from {url!r}")

def extract_sheet_id(url: str) -> str:
    """Pull the Spreadsheet ID from its URL."""
    m = re.search(r"/spreadsheets/d/([A-Za-z0-9_-]+)", url)
    if m:
        print("Sheet ID:", m.group(1))
        return m.group(1)
    raise ValueError(f"Could not parse sheet ID from {url!r}")

def extract_gdrive_id(url: str) -> str:
    """Pull the Google Drive file ID from its URL."""
    patterns = [
        r"/file/d/([A-Za-z0-9_-]+)",
        r"/drive/([A-Za-z0-9_-]+)",
        r"id=([A-Za-z0-9_-]+)",
        r"^([A-Za-z0-9_-]+)$"  # For direct IDs
    ]
    for pat in patterns:
        m = re.search(pat, url)
        if m:
            return m.group(1)
    return None

def list_notebooks_recursive(folder_id: str) -> list[dict]:
    """
    Recursively list every .ipynb under `folder_id`, including shared-drive.
    Returns a flat list of {id, name} dicts.
    """
    notebooks = []
    query = f"'{folder_id}' in parents and trashed = false"
    page_token = None

    while True:
        resp = drive_service.files().list(
            q=query,
            spaces='drive',
            fields="nextPageToken, files(id, name, mimeType)",
            pageSize=500,
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            corpora='allDrives',
            pageToken=page_token
        ).execute()

        for f in resp.get('files', []):
            mtype = f['mimeType']
            # folder → recurse
            if mtype == 'application/vnd.google-apps.folder':
                notebooks.extend(list_notebooks_recursive(f['id']))

            # notebook → collect (.ipynb files or Google Colaboratory files)

            # Colab notebooks are typically stored as JSON or with specific ID formats
            # We'll check for .ipynb extension or if it's a JSON file that might be a notebook
            elif (f['name'].lower().endswith('.ipynb') or
                  ('colab' in f.get('metadata', ''))):
                notebooks.append({'id': f['id'], 'name': f['name']})

        page_token = resp.get('nextPageToken')
        if not page_token:
            break

    return notebooks

def get_file_metadata(file_id: str) -> dict:
    """Get metadata for a single file by ID"""
    try:
        return drive_service.files().get(
            fileId=file_id,
            fields="id,name,mimeType",
            supportsAllDrives=True,
        ).execute()
    except HttpError:
        return None

# Determine if we're using direct URLs or a folder search
url_dict = {}

if urls and len(urls) > 0:
    # Using direct URLs, extract file IDs and get metadata
    print(f"Processing {len(urls)} supplied URLs...")
    for url in urls:
        file_id = extract_gdrive_id(url)
        if file_id:
            metadata = get_file_metadata(file_id)
            if metadata and metadata.get('name', '').lower().endswith('.ipynb'):
                name = metadata.get('name')
                url_dict[name] = f"https://colab.research.google.com/drive/{file_id}"

    print(f"Found {len(url_dict)} notebooks:")
    for name in url_dict:
        print(f" • {name}")

else:
    # Extract the root folder ID
    root_folder_id = extract_folder_id(FOLDER_URL)

    # Fetch every notebook under it
    all_notebooks = list_notebooks_recursive(root_folder_id)
    print(f"Found {len(all_notebooks)} notebooks in the specified folder")

    # Apply include/exclude filters on the lowercase name
    filtered = []
    include_kwds_lc = [kw.lower() for kw in INCLUDE_KWDS]
    exclude_kwds_lc = [kw.lower() for kw in EXCLUDE_KWDS]

    for f in all_notebooks:
        name_lc = f['name'].lower()

        # Check include keywords (if any provided)
        include_match = not include_kwds_lc or any(kw in name_lc for kw in include_kwds_lc)

        # Check exclude keywords
        exclude_match = any(kw in name_lc for kw in exclude_kwds_lc)

        if include_match and not exclude_match:
            filtered.append(f)

    # Build your url_dict
    url_dict = {
        nb['name']: f"https://colab.research.google.com/drive/{nb['id']}"
        for nb in filtered
    }

    print(f"Found {len(url_dict)} matching notebooks:")
    for name in url_dict:
        print(f" • {name}")

Found 12 notebooks in the specified folder
Found 10 matching notebooks:
 • [REVISING] S4 - 10 - Revision (and the future).ipynb
 • [REVISING] S4 - 9 - Time Complexity.ipynb
 • [REVISING] S4 - 8 - Debugging Methods and Security.ipynb
 • [REVISED] S4 - 7 - Refactoring and Variable Names (and Networks).ipynb
 • [REVISED] S4 - 6 - OOP continued #3 Inheritance and Graphics.ipynb
 • [REVISING] S4 - 5 - Variable Scopes and Mutability.ipynb
 • [REVISING] S^4 Week 3 Breakouts: OOP Fundamentals and Programming Languages.ipynb
 • [REVISING] S^4 Week 4 Breakouts: OOP Functions and Software Engineering.ipynb
 • [REVISING] S^4 Week 2 Breakouts: Recursion, Memory, and Time.ipynb
 • S4 - 1 - Integers and Floats in Binary.ipynb


In [8]:
# Now that we have the url_dict, get comments from each notebook
def get_all_comments(file_id: str) -> list[dict]:
    """Page through Drive API to collect every comment on a file."""
    comments = []
    page_token = None
    while True:
        try:
            resp = drive_service.comments().list(
                fileId=file_id,
                fields="nextPageToken,comments(id,content,author(displayName),modifiedTime)",
                pageToken=page_token,
                includeDeleted=False
            ).execute()
            comments.extend(resp.get('comments', []))
            page_token = resp.get('nextPageToken')
            if not page_token:
                break
        except HttpError as e:
            print(f"Error getting comments for {file_id}: {e}")
            break
    return comments

def append_rows_to_sheet(spreadsheet_id: str, rows: list[list]):
    """Batch‐append rows to the target sheet."""
    body = {'values': rows}
    sheets_service.spreadsheets().values().append(
        spreadsheetId=spreadsheet_id,
        range=TARGET_RANGE,
        valueInputOption='USER_ENTERED',
        insertDataOption='INSERT_ROWS',
        body=body
    ).execute()

# ─── Main ─────────────────────────────────────────────────────────────────

def main():
    # Extract the spreadsheet ID
    spreadsheet_id = extract_sheet_id(TARGET_SHEET_URL)

    # Header + first append
    header = [["Source", "Author", "Comment", "Modified Time", "Link"]]
    append_rows_to_sheet(spreadsheet_id, header)

    # Loop through notebooks
    for nb_name, nb_url in url_dict.items():
        file_id = extract_gdrive_id(nb_url)
        if not file_id:
            print(f"✖︎ Could not parse ID from URL for '{nb_name}'")
            continue

        try:
            comments = get_all_comments(file_id)
            rows = []
            for c in comments:
                cid     = c.get('id')
                author  = c.get('author', {}).get('displayName', '(unknown)')
                content = c.get('content', '')
                mtime   = c.get('modifiedTime')
                link    = f"{nb_url}?commentId={cid}"
                rows.append([nb_name, author, content, mtime, link])

            if rows:
                append_rows_to_sheet(spreadsheet_id, rows)
            print(f"✔︎ Pulled {len(rows)} comments for '{nb_name}'")

        except HttpError as e:
            print(f"✖︎ Drive API error for '{nb_name}': {e}")

    print("✅ All done.")

# Execute the main function
if __name__ == "__main__":
    main()

Sheet ID: 1V-huGRgju495WbUBchVC_XBUD--YDTPHHq6b0GaTJxY
✔︎ Pulled 8 comments for '[REVISING] S4 - 10 - Revision (and the future).ipynb'
✔︎ Pulled 9 comments for '[REVISING] S4 - 9 - Time Complexity.ipynb'
✔︎ Pulled 9 comments for '[REVISING] S4 - 8 - Debugging Methods and Security.ipynb'
✔︎ Pulled 9 comments for '[REVISED] S4 - 7 - Refactoring and Variable Names (and Networks).ipynb'
✔︎ Pulled 8 comments for '[REVISED] S4 - 6 - OOP continued #3 Inheritance and Graphics.ipynb'
✔︎ Pulled 8 comments for '[REVISING] S4 - 5 - Variable Scopes and Mutability.ipynb'
✔︎ Pulled 8 comments for '[REVISING] S^4 Week 3 Breakouts: OOP Fundamentals and Programming Languages.ipynb'
✔︎ Pulled 17 comments for '[REVISING] S^4 Week 4 Breakouts: OOP Functions and Software Engineering.ipynb'
✔︎ Pulled 12 comments for '[REVISING] S^4 Week 2 Breakouts: Recursion, Memory, and Time.ipynb'
✔︎ Pulled 13 comments for 'S4 - 1 - Integers and Floats in Binary.ipynb'
✅ All done.
